## Расчет нагрузки на систему


## Общие сведения
После сбора первичных требований к системе очень важно оценить необходимую инфраструктуру. 
Оценка позволит скорректировать ожидания от продукта.

## Основные метрики расчета
1.  **Пользовательский трафик**: Оценка аудитории (DAU/MAU), время работы, объем контента.
2.  **Сетевая нагрузка**: Удерживаемые соединения и пропускная способность (bandwidth).
3.  **Вычислительная нагрузка**: RPS (Requests Per Second).
4.  **Нагрузка на хранилище**: Объем данных и его прирост.

---

### Справочные значения производительности
*   **Соединения**: Современный сервер держит 10k-100k соединений.
*   **Сеть**: 1GbE (обычное облако) - 40GbE+ (оптоволокно).
*   **RPS (Requests Per Second)**:
    *   Простые сценарии (текст): ~100k RPS (облако) / ~500k RPS (физика).
    *   С БД (чтение): ~10k RPS (облако) / ~50k RPS (физика).
    *   С БД (запись): ~1k RPS (облако) / ~5k RPS (физика).
*   **Хранилище**:
    *   **HDD**: 100-300 MB/s, $30/TB, до 200TB на сервер.
    *   **SSD (NVMe)**: 3-5 GB/s, $300/TB, до 50TB на сервер.
    *   **RAM**: 50 GB/s, $10,000/TB, до 1TB на сервер.


## Практические примеры расчета

### 1. Сокращатель ссылок


**Допущения:**
*   100M сокращений каждый месяц.
*   MAU 10M, DAU 1M (из текста, хотя 100M/мес / 1M DAU = 3.3 сокращения в день).
*   Соотношение чтения и записи 100 к 1.
*   Размер одной записи: 1 KB.
*   Горизонт хранения: 5 лет.


In [1]:

# Параметры
monthly_writes = 100_000_000  # 100 миллионов новых ссылок в месяц
read_write_ratio = 100        # На 1 создание приходится 100 переходов
record_size_kb = 1            # Размер одной записи в КБ
retention_years = 5           # Храним 5 лет

# Константы
SECONDS_IN_MONTH = 30 * 24 * 60 * 60
KB_TO_MB = 1 / 1024
KB_TO_TB = 1 / (1024 * 1024 * 1024)

# 1. Расчет RPS (Requests Per Second)
write_rps = monthly_writes / SECONDS_IN_MONTH
read_rps = write_rps * read_write_ratio

print(f"--- Вычислительная нагрузка ---")
print(f"Write RPS: {write_rps:.2f} (Записей в секунду)")
print(f"Read RPS:  {read_rps:.2f} (Чтений в секунду)")

# 2. Расчет Сетевого трафика (Bandwidth)
# Трафик генерируется в основном на чтение
bandwidth_kbps = read_rps * record_size_kb
bandwidth_mbps = bandwidth_kbps * KB_TO_MB
bandwidth_mbits = bandwidth_mbps * 8 # bits

print(f"\n--- Сетевая нагрузка ---")
print(f"Исходящий трафик: {bandwidth_mbps:.2f} MB/s ({bandwidth_mbits:.2f} Mbps)")

# 3. Расчет Хранилища (Storage)
total_records = monthly_writes * 12 * retention_years
total_storage_tb = total_records * record_size_kb * KB_TO_TB

print(f"\n--- Хранилище (за {retention_years} лет) ---")
print(f"Всего записей: {total_records/1_000_000_000:.1f} млрд")
print(f"Требуемое место: {total_storage_tb:.2f} TB")


--- Вычислительная нагрузка ---
Write RPS: 38.58 (Записей в секунду)
Read RPS:  3858.02 (Чтений в секунду)

--- Сетевая нагрузка ---
Исходящий трафик: 3.77 MB/s (30.14 Mbps)

--- Хранилище (за 5 лет) ---
Всего записей: 6.0 млрд
Требуемое место: 5.59 TB


### 2. Хостинг текстов


**Допущения:**
*   1M загрузок каждый день (MAU 10M, DAU 500K, 2 загрузки в среднем).
*   Соотношение чтения и записи 10 к 1.
*   Размер записи: 10 KB.
*   Горизонт: 5 лет.


In [ ]:

daily_writes = 1_000_000
read_write_ratio = 10
record_size_kb = 10
retention_years = 5

SECONDS_IN_DAY = 24 * 60 * 60

# RPS
write_rps = daily_writes / SECONDS_IN_DAY
read_rps = write_rps * read_write_ratio

print(f"Write RPS: {write_rps:.2f}")
print(f"Read RPS:  {read_rps:.2f}")

# Traffic
bandwidth_mbps = (read_rps * record_size_kb) / 1024
print(f"Bandwidth: {bandwidth_mbps:.2f} MB/s ({bandwidth_mbps*8:.2f} Mbps)")

# Storage
total_records = daily_writes * 365 * retention_years
total_storage_tb = (total_records * record_size_kb) / (1024 * 1024 * 1024)

print(f"Total Records (5 years): {total_records/1_000_000_000:.1f} B")
print(f"Total Storage: {total_storage_tb:.2f} TB")


### 3. Автодополнение


**Допущения:**
*   1 миллиард обращений к сервису в день.
*   10 миллионов уникальных запросов покрывают большую часть.
*   Запрос: 5 слов по 10 символов = 50 байт (округлим до 200 байт).
*   Прирост уникальных запросов: 5% новых каждый день.


In [ ]:

daily_requests = 1_000_000_000
unique_requests_base = 10_000_000
request_size_bytes = 200
new_unique_percent = 0.05
years = 5

# Storage for daily cache
daily_storage_gb = (unique_requests_base * request_size_bytes) / (1024 * 1024 * 1024)
print(f"Daily Cache Storage: {daily_storage_gb:.2f} GB")

# Growth over 5 years
days = years * 365
total_new_unique = days * (unique_requests_base * new_unique_percent)
total_index_size_gb = daily_storage_gb + (total_new_unique * request_size_bytes / (1024 * 1024 * 1024))

print(f"Total Index Size (5 years): {total_index_size_gb:.2f} GB")


### 4. Облачный диск


**Допущения:**
*   1 млрд пользователей всего, 100M DAU.
*   У каждого 5 устройств.
*   У каждого 100 файлов по 100 KB, активные меняют 1 файл в день.


In [ ]:

dau = 100_000_000
files_changed_daily = 1
file_size_kb = 100
devices_per_user = 5

# Network (Sync)
# 1 change -> upload (1) + download to other devices (4)
traffic_per_user_kb = files_changed_daily * file_size_kb * devices_per_user 
total_daily_traffic_tb = (dau * traffic_per_user_kb) / (1024 * 1024 * 1024)

print(f"Daily Traffic: {total_daily_traffic_tb:.2f} TB")
print(f"Bandwidth: {(total_daily_traffic_tb * 1024 * 1024 * 8) / (24*3600):.2f} Gbps")

# Storage
total_users = 1_000_000_000
avg_files = 100
total_storage_pb = (total_users * avg_files * file_size_kb) / (1024 * 1024 * 1024 * 1024)

print(f"Total Storage Required: {total_storage_pb:.2f} PB")
